In [1]:
import cv2
import numpy as np
import pandas as pd

In [2]:
# 신경망 모델 구조 파일
cfg_file = './opencv_data/yolo/yolov3.cfg'
# 가중치 파일
weights_file = './opencv_data/yolo/yolov3.weights'
# 인식 가능한 사물의 이름
class_file = './opencv_data/yolo/coco.names'

In [3]:
# yolo 모델을 복원한다.
net = cv2.dnn.readNet(weights_file, cfg_file)
net

< cv2.dnn.Net 00000241581303B0>

In [4]:
# 물체 종류를 리스트로 추출한다.
with open(class_file, 'rt') as fp:
    classes = fp.readlines()
    classes = [line.strip() for line in classes]
    
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [5]:
# 각 물체를 의미하는 색상은 랜덤하게 생성한다.
colors = np.random.uniform(0, 255, size=(len(classes), 3))
colors

array([[239.18909205,  19.58324233, 232.40853988],
       [201.19689386, 114.33937754,  67.71519739],
       [127.8734679 , 192.19784791,  27.44518295],
       [196.91194282, 189.77106189,  69.8988682 ],
       [188.37205593, 146.86460551, 241.05716293],
       [203.49600117, 117.70504353, 242.39883075],
       [190.66810434,  73.51712236, 153.63804656],
       [155.10042983, 138.89462372,  25.28971551],
       [ 16.19351589, 253.49435766,   8.48776353],
       [ 34.92003807, 180.43693842,  52.69754438],
       [ 87.67865891,  24.74220763,  70.14153871],
       [ 54.89461576,  61.48760066, 110.47758536],
       [ 66.87067625,  51.34995797, 219.8488899 ],
       [201.64506515,  20.29431247, 183.47898646],
       [ 18.73783173,  24.53697704, 163.49848941],
       [ 86.28305147, 105.56414947, 138.34716199],
       [193.30333153, 103.25199981,  42.07528363],
       [171.53209144, 129.68644174, 101.25499735],
       [191.19545897, 117.83351522, 119.9363934 ],
       [ 91.0942628 , 142.30977

In [6]:
# 영상 데이터를 가져온다.
cap = cv2.VideoCapture('./opencv_data/video/yolo_01.mp4')

In [7]:
# 엔터키를 누를 때 까지 반복한다.
while cv2.waitKey(33) < 0 :
    # 현재 프레임을 가져온다.
    ret, img = cap.read()
    # 더이상 가져올 것이 없다면(끝까지 갔을 경우)
    if img is None :
        break
        
    # 이미지의 세로, 가로, 채널
    height, width, channel = img.shape
    
    # 이미지의 크기를 조정한다.
    # 16*16 부터 16 배수로 설정해주면 된다.(강제)
    a1 = cv2.resize(img, (416, 416))
    
    # 2진 데이터로 변환한다.
    # 0.00392 - 임계값. 이 값보다 크면 1, 작으면 0으로 변환한다.
    # 이 값은 완전 흰색이 1, 완전 검정색을 0으로 취급한다.
    # 여기서 설정하는 임계값은 yolo에서 추천하는 값
    blob = cv2.dnn.blobFromImage(a1, 0.00392, (416, 416), (0, 0, 0))

    # 출력층의 이름
    output_layers = net.getUnconnectedOutLayersNames()
    output_layers
    
    # 이미지 데이터를 모델에 셋팅해준다.
    net.setInput(blob)
    
    # 물체를 검출한다.
    outs = net.forward(output_layers)
    
    # 예측된 결과를 담을 리스트
    class_id_list = []
    # 예측 정확도
    confidence_list = []
    # 인지된 사물의 영역
    box_list = []

    # 확률의 임계값
    # 이 확률 이상인 것만 결과에 담는다.
    confidence_limit = 0.5

    # 출력층의 수 만큼 반복한다.
    for out in outs :
        # 현재의 출력층이 검출한 물체의 수 만큼 반복한다.
        for detection in out :
            # 원핫 인코딩되어 있는 확률 값만 가져온다.
            score_list = detection[5:]
            # 확률이 가장 높은 물체 값을 가져온다.
            class_id = np.argmax(score_list)
            # 확률을 가져온다.
            confidence = score_list[class_id]

            # print(classes[class_id], score_list[class_id])

            # 확률이 확률 임계값 이상인 것만 처리한다.
            if confidence >= confidence_limit :
                # 물체의 좌표를 계산한다.
                # 예측 결과에서 앞의 4개는
                # 중심점 x 비율, 중심점 y 비율, 가로 비율, 세로 비율이다.
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)

                # 가로길이
                w = int(detection[2] * width)
                # 세로길이
                h = int(detection[3] * height)
                # 좌측 상단 x
                x = int(center_x - w / 2 )
                # 좌측 상단 y
                y = int(center_y - h / 2)

                # 담는다.
                box_list.append([x, y, w, h])
                confidence_list.append(float(confidence))
                class_id_list.append(class_id)

    # IoU : 두 면적이 중첩되는 영역의 넓이를 두 면적의 합친 총 면적으로 나눈 값
    # 두 면적이 얼마나 중첩되어 있는지 평가할 수 있는 지표가 된다.
    # 0~1 사이가 나오며 값이 클수록 중첩된 부분이 많다고 평가할 수 있다.

    # NMS : IoU 방식으로 면적을 평가하고 중첩이 많이 됬다고 판단되는 영역을 제거하는 방식.
    # IoU 방식으로 평가하여 중첩잉 많이 됬다고 판단되는 영역을 모두 아우를 수 있는 영역을 만들어낸다.
    # 0.4 : 확인하지 않고 버릴 영역의 확률값

    # 결과는 인식한 사물의 index 번호
    indexes = cv2.dnn.NMSBoxes(box_list, confidence_list, confidence_limit, 0.4)
    
    # 결과를 토대로 그린다.
    # 검출된 물체의 수 만큼 반복한다.
    for i in range(len(box_list)):
        # NMS를 통해 추출한 영역만 표시한다.
        if i in indexes:

            # 좌표값을 추출한다.
            x, y, w, h = box_list[i]
            # 이름
            label = str(classes[class_id_list[i]])
            # 색상
            color = colors[class_id_list[i]]

            # 네모를 그린다.
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            # 이름을 표시한다.
            cv2.putText(img, label, (x, y-30), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

    cv2.imshow('Object Detection', img)
cv2.waitKey()
cv2.destroyAllWindows()